In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import datetime
train = pd.read_csv('./train.csv')
sub = pd.read_csv('./sample_submission.csv')
monthly = train.groupby(['item_id', 'year', 'month']).agg({
    'value': 'sum',
    'weight': 'sum',
    'quantity': 'sum',
    'hs4': 'first'
}).reset_index()

monthly['year_month'] = pd.to_datetime(
    monthly['year'].astype(str) + '-' + monthly['month'].astype(str),
    format='%Y-%m'
)

pivot_value = monthly.pivot_table(
    index='year_month',
    columns='item_id',
    values='value',
    fill_value=0
).sort_index()

# Transpose: item을 index로 (예시 코드 형식)
pivot = pivot_value.T  # (100 items × 43 months)

pivot.head(5)
item_to_hs4 = monthly.groupby('item_id')['hs4'].first().to_dict()
from statsmodels.tsa.seasonal import STL

# ==============================
# STL residual pivot 만들기
# ==============================

def extract_residual(series, period=12):
    """
    pivot.loc[item_id] 1개 시계열을 받아 residual만 반환
    """
    s = pd.Series(series)

    # 0 → NaN 처리 후 보간
    s = s.replace(0, np.nan).interpolate().fillna(method="bfill").fillna(method="ffill")

    stl = STL(s, period=period, robust=True).fit()
    return stl.resid

# residual pivot 생성
residual_pivot = pd.DataFrame(
    {
        item: extract_residual(pivot.loc[item])
        for item in pivot.index
    }
).T

print("원본 pivot:", pivot.shape)
print("Residual pivot:", residual_pivot.shape)
date_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

pivot.to_csv(f"hs_pivot_{date_str}.csv",index=None,encoding='utf-8-sig')
residual_pivot.to_csv(f"hs_residual_pivot_{date_str}.csv",index=None,encoding='utf-8-sig')
residual_pivot.head()
from statsmodels.tsa.seasonal import seasonal_decompose

# --- 월 리스트 (나중에 Step 7에서도 씀) ---
months_dt = pivot.columns.to_list()

# === STL 분해 기반 trend_pivot, seasonal_pivot 생성 ===
trend_data = {}
seasonal_data = {}

for item_id in pivot.index:
    series = pivot.loc[item_id].astype(float)
    series_clean = series.fillna(0)

    try:
        decomposition = seasonal_decompose(series_clean, model='additive', period=12)
        trend_data[item_id] = decomposition.trend
        seasonal_data[item_id] = decomposition.seasonal
    except Exception as e:
        # 실패 시 NaN으로 채움
        trend_data[item_id] = pd.Series([np.nan] * len(series_clean), index=series_clean.index)
        seasonal_data[item_id] = pd.Series([np.nan] * len(series_clean), index=series_clean.index)

trend_pivot = pd.DataFrame(trend_data).T
seasonal_pivot = pd.DataFrame(seasonal_data).T

print("trend_pivot 및 seasonal_pivot 생성 완료.")
from scipy.stats import pearsonr

def safe_corr_with_pvalue(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0, 1.0
    corr, p_value = pearsonr(x, y)
    return float(corr), float(p_value) # 피어슨 상관계수, p-value
from statsmodels.tsa.stattools import grangercausalitytests

def granger_test(x, y, max_lag=6):
    """
    x가 y를 Granger-cause 하는지 검증
    """
    try:
        data = pd.DataFrame({'x': x, 'y': y})
        test_result = grangercausalitytests(data[['y', 'x']],
                                            maxlag=max_lag,
                                            verbose=False)
        # 각 lag의 p-value 중 최소값 반환
        p_values = [test_result[lag][0]['ssr_ftest'][1]
                   for lag in range(1, max_lag+1)]
        return min(p_values)
    except:
        return 1.0
def calculate_score_advanced(corr, p_value, granger_p, lag):
    """
    정교한 공행성 점수 계산
    """

    # 1. 상관계수 점수 (0~50점) - 비선형 스케일링
    # 강한 상관관계에 더 높은 보상
    if abs(corr) >= 0.8:
        corr_score = 50
    elif abs(corr) >= 0.7:
        corr_score = 45
    elif abs(corr) >= 0.6:
        corr_score = 38
    elif abs(corr) >= 0.5:
        corr_score = 30
    else:
        corr_score = abs(corr) * 50  # 0.5 미만은 선형

    # 2. 통계적 유의성 점수 (0~25점) - 더 세분화
    if p_value < 0.001:
        sig_score = 25  # 매우 강한 유의성
    elif p_value < 0.01:
        sig_score = 20
    elif p_value < 0.05:
        sig_score = 12
    else:
        sig_score = 5  # 약한 유의성도 약간 점수

    # 3. Granger 인과성 점수 (0~20점) - 더 세분화
    if granger_p < 0.001:
        granger_score = 20  # 매우 강한 인과성
    elif granger_p < 0.01:
        granger_score = 16
    elif granger_p < 0.05:
        granger_score = 10
    elif granger_p < 0.10:
        granger_score = 5  # 약한 인과성
    else:
        granger_score = 0

    # 4. Lag 점수 (0~5점) - 비선형, 너무 짧거나 긴 lag 패널티
    # 최적 lag는 2-3개월로 가정
    if lag == 2 or lag == 3:
        lag_score = 5  # 최적
    elif lag == 1 or lag == 4:
        lag_score = 4  # 양호
    elif lag == 5:
        lag_score = 2  # 보통
    else:  # lag >= 6
        lag_score = 0  # 너무 김

    total_score = corr_score + sig_score + granger_score + lag_score

    return total_score
from tqdm import tqdm

def find_comovement_pairs(
    pivot,
    max_lag=6,
    min_nonzero=12,
    corr_threshold=0.4,
    score_threshold=40
):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)
    results = []

    for i, leader in tqdm(enumerate(items), total=len(items)):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        candidates = []

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0
            best_p_value = 1.0

            # lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue

                corr, p_value = safe_corr_with_pvalue(x[:-lag], y[lag:])

                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag
                    best_p_value = p_value

            # 기본 임계값 통과 시
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                # Granger test
                granger_p = granger_test(x, y, max_lag=best_lag)

                # 종합 점수 계산
                score = calculate_score_advanced(
                    best_corr, best_p_value, granger_p, best_lag
                )

                # 점수 임계값 통과 시 후보에 추가
                if score >= score_threshold:
                    candidates.append({
                        "following_item_id": follower,
                        "best_lag": best_lag,
                        "max_corr": best_corr,
                        "p_value": best_p_value,
                        "granger_p_value": granger_p,
                        "comovement_score": score,
                    })

        # 알파벳 순으로 정렬
        candidates.sort(key=lambda x: x['following_item_id'])

        for candidate in candidates:
            results.append({
                "leading_item_id": leader,
                "following_item_id": candidate["following_item_id"],
                "best_lag": candidate["best_lag"],
                "max_corr": candidate["max_corr"]
            })

    pairs = pd.DataFrame(results)
    return pairs

# # 실행
# pairs = find_comovement_pairs(pivot,9,12,0.4,35)
# print("탐색된 공행성쌍 수:", len(pairs))
# pairs.head(20)
import matplotlib.pyplot as plt

# (A) raw plot
def plot_raw_pair(item_a, item_b, pivot):
    plt.figure(figsize=(10,4))
    plt.plot(pivot.columns, pivot.loc[item_a], marker='o', label=f"{item_a} (raw)")
    plt.plot(pivot.columns, pivot.loc[item_b], marker='o', label=f"{item_b} (raw)")
    plt.title(f"[RAW] {item_a} → {item_b}")
    plt.grid()
    plt.legend()
    plt.tight_layout()
    plt.show()

# (B) residual plot
def plot_resid_pair(item_a, item_b, residual_pivot):
    plt.figure(figsize=(10,4))
    plt.plot(residual_pivot.columns, residual_pivot.loc[item_a], marker='o',
             label=f"{item_a} (residual)")
    plt.plot(residual_pivot.columns, residual_pivot.loc[item_b], marker='o',
             label=f"{item_b} (residual)")
    plt.title(f"[RESIDUAL] {item_a} → {item_b}")
    plt.grid()
    plt.legend()
    plt.tight_layout()
    plt.show()

#(C) raw + residual 둘 다 한 번에 보여주는 plot 함수
def plot_raw_and_residual_save(a, b, pivot, residual_pivot, save_dir):
    """
    raw + residual 두 개의 개별 plot을 하나의 파일로 저장
    """

    # 폴더 없으면 생성
    os.makedirs(save_dir, exist_ok=True)

    fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

    # 1) 원본 raw plot
    axes[0].plot(pivot.columns, pivot.loc[a], marker='o', label=f'{a} (raw)')
    axes[0].plot(pivot.columns, pivot.loc[b], marker='o', label=f'{b} (raw)')
    axes[0].set_title(f'[RAW] {a} vs {b}')
    axes[0].grid(True)
    axes[0].legend()

    # 2) residual plot
    axes[1].plot(residual_pivot.columns, residual_pivot.loc[a], marker='o',
                 label=f'{a} (residual)')
    axes[1].plot(residual_pivot.columns, residual_pivot.loc[b], marker='o',
                 label=f'{b} (residual)')
    axes[1].set_title(f'[RESIDUAL] {a} vs {b}')
    axes[1].grid(True)
    axes[1].legend()

    plt.tight_layout()

    # 저장 파일명
    file_name = f"{a}__{b}.png"
    save_path = os.path.join(save_dir, file_name)

    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"저장 완료 → {save_path}")

pairs = find_comovement_pairs(pivot, 9, 12, 0.35, 45)
pairs_resid = find_comovement_pairs(residual_pivot, 9, 12, 0.35, 45)
print("탐색된 공행성쌍 수 (raw):", len(pairs))
print("탐색된 공행성쌍 수 (resid):", len(pairs_resid))
pairs.to_csv(f"hs_pairs_{date_str}.csv",encoding='utf-8-sig',index=None)
pairs_resid.to_csv(f"hs_pairs_resid_{date_str}.csv",encoding='utf-8-sig',index=None)
import os

SAVE_DIR = "plots_raw_pairs"   # 원하는 폴더명

# 🔥 raw 기준 pairs를 plot
N = 20  # 원하는 개수만큼 plot

for i in range(min(N, len(pairs))):
    row = pairs.iloc[i]
    a = row["leading_item_id"]
    b = row["following_item_id"]

    plot_raw_and_residual_save(a, b, pivot, residual_pivot, SAVE_DIR)
# 🚀 4. (옵션) 두 결과 차이 비교 (교집합/차집합)
raw_set = set((r.leading_item_id, r.following_item_id) for _, r in pairs.iterrows())
resid_set = set((r.leading_item_id, r.following_item_id) for _, r in pairs_resid.iterrows())

common_pairs = raw_set & resid_set
raw_only = raw_set - resid_set
resid_only = resid_set - raw_set

print("공통쌍:", len(common_pairs))
print("raw only:", len(raw_only))
print("residual only:", len(resid_only))
# 🔹 여기서부터: df_meta 대신, pairs_raw만 사용 + f_is_common만 추가
# f_is_common: raw & resid 모두에 등장하는 교집합 쌍이면 1, 아니면 0
pairs['f_is_common'] = pairs.apply(
    lambda r: 1 if (r['leading_item_id'], r['following_item_id']) in common_pairs else 0,
    axis=1
)

print("pairs에 f_is_common 추가 완료. shape:", pairs.shape)
# ============================================================
# 7. DTW / volatility_corr / Granger 등 메타피처 생성
# ============================================================

print("\n===== 메타피처 생성 시작 =====")

PairsMeta_rows = []

for idx, row in tqdm(pairs.iterrows(), total=len(pairs)):
    leader = row["leading_item_id"]
    follower = row["following_item_id"]
    lag = int(row["best_lag"])
    corr_raw = float(row["max_corr"])

    # --------------------------
    # 1. 시계열 준비
    # --------------------------
    ts_L_raw = pivot.loc[leader].fillna(0).values
    ts_F_raw = pivot.loc[follower].fillna(0).values

    ts_L_res = residual_pivot.loc[leader]
    ts_F_res = residual_pivot.loc[follower]

    # --------------------------
    # 2. DTW (lag 반영)
    # --------------------------
    dtw_dist = np.nan
    if lag > 0 and lag < len(ts_L_raw):
        try:
            dtw_dist = dtw(ts_L_raw[:-lag], ts_F_raw[lag:])
        except:
            pass

    # --------------------------
    # 3. Granger p-value
    # --------------------------
    granger_p = np.nan
    try:
        df_g = pd.DataFrame({"F": ts_F_raw, "L": ts_L_raw})
        if lag > 0 and len(df_g) > lag + 5:
            result = grangercausalitytests(df_g[["F", "L"]], maxlag=[lag], verbose=False)
            granger_p = result[lag][0]["ssr_ftest"][1]
    except:
        pass

    # --------------------------
    # 4. residual volatility corr
    # --------------------------
    vol_corr = np.nan
    try:
        L_vol = ts_L_res.rolling(window=3).std().fillna(0)
        F_vol = ts_F_res.rolling(window=3).std().fillna(0)
        vol_corr = L_vol.corr(F_vol)
    except:
        pass

    # --------------------------
    # 5. Corr scaling
    # --------------------------
    corr_scaled = np.log1p(abs(corr_raw)) * np.sign(corr_raw)

    # --------------------------
    # 하나의 메타 row 저장
    # --------------------------
    PairsMeta_rows.append({
        "leading_item_id": leader,
        "following_item_id": follower,

        # 원본 pair meta
        "f_corr_raw": corr_raw,
        "f_corr_scaled": corr_scaled,
        "f_lag": lag,
        "f_is_common": row["f_is_common"],

        # 새로 생성된 메타피처
        "f_dtw_distance_raw": dtw_dist,
        "f_granger_pvalue": granger_p,
        "f_volatility_corr_resid": vol_corr,
    })


# 메타피처 테이블 완성
date_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
pairs_meta = pd.DataFrame(PairsMeta_rows)
print("pairs_meta 생성 완료 →", pairs_meta.shape)

# 저장(optional)
pairs_meta.to_csv(f"hs_pairs_meta_{date_str}.csv", index=False, encoding="utf-8-sig")

현재 문제점 : 단일 임계값으로 페어 필터링, 모든 품목에 동일한 기준 적용, HS4 코드 정보를 활용하지 않음
해볼 수 있는 시도들로는 hs4 정보 활용한 페어쌍 탐색, Residual상관계수 반영한 종합 점수, 트렌드/계절성 분해 피처 추가, 최근 데이터 가중치 높인 Dynamic 피처, 적응적 임계값 적용
일단 이중에서 1번을 적용한 코드

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import datetime
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL, seasonal_decompose
from statsmodels.tsa.stattools import grangercausalitytests
from tqdm import tqdm
from lightgbm import LGBMRegressor

# dtw + shap
try:
    from tslearn.metrics import dtw
    import shap
except:
    print("Warning: tslearn or shap not installed")

# ============================================================
# 1. 데이터 로드 및 전처리
# ============================================================
train = pd.read_csv('./train.csv')
sub = pd.read_csv('./sample_submission.csv')

monthly = train.groupby(['item_id', 'year', 'month']).agg({
    'value': 'sum',
    'weight': 'sum',
    'quantity': 'sum',
    'hs4': 'first'
}).reset_index()

monthly['year_month'] = pd.to_datetime(
    monthly['year'].astype(str) + '-' + monthly['month'].astype(str),
    format='%Y-%m'
)

pivot_value = monthly.pivot_table(
    index='year_month',
    columns='item_id',
    values='value',
    fill_value=0
).sort_index()

# Transpose: item을 index로
pivot = pivot_value.T  # (100 items × 43 months)

print("원본 pivot shape:", pivot.shape)
pivot.head(5)

item_to_hs4 = monthly.groupby('item_id')['hs4'].first().to_dict()

# ============================================================
# 2. STL 분해 - Residual, Trend, Seasonal
# ============================================================

def extract_residual(series, period=12):
    """
    pivot.loc[item_id] 1개 시계열을 받아 residual만 반환
    """
    s = pd.Series(series)
    # 0 → NaN 처리 후 보간
    s = s.replace(0, np.nan).interpolate(method='linear').fillna(method="bfill").fillna(method="ffill")
    stl = STL(s, period=period, robust=True).fit()
    return stl.resid

# residual pivot 생성
residual_pivot = pd.DataFrame(
    {
        item: extract_residual(pivot.loc[item])
        for item in pivot.index
    }
).T

print("Residual pivot shape:", residual_pivot.shape)

# 날짜 문자열
date_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# CSV 저장
pivot.to_csv(f"hs_pivot_{date_str}.csv", index=True, encoding='utf-8-sig')
residual_pivot.to_csv(f"hs_residual_pivot_{date_str}.csv", index=True, encoding='utf-8-sig')

# --- 월 리스트 ---
months_dt = pivot.columns.to_list()

# === Trend, Seasonal pivot 생성 ===
trend_data = {}
seasonal_data = {}

for item_id in pivot.index:
    series = pivot.loc[item_id].astype(float)
    series_clean = series.fillna(0)

    try:
        decomposition = seasonal_decompose(series_clean, model='additive', period=12, extrapolate_trend='freq')
        trend_data[item_id] = decomposition.trend
        seasonal_data[item_id] = decomposition.seasonal
    except Exception as e:
        # 실패 시 NaN으로 채움
        trend_data[item_id] = pd.Series([np.nan] * len(series_clean), index=series_clean.index)
        seasonal_data[item_id] = pd.Series([np.nan] * len(series_clean), index=series_clean.index)

trend_pivot = pd.DataFrame(trend_data).T
seasonal_pivot = pd.DataFrame(seasonal_data).T

print("trend_pivot 및 seasonal_pivot 생성 완료.")

# ============================================================
# 3. 공행성 탐색 함수들
# ============================================================

def safe_corr_with_pvalue(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0, 1.0
    corr, p_value = pearsonr(x, y)
    return float(corr), float(p_value)

def granger_test(x, y, max_lag=6):
    """
    x가 y를 Granger-cause 하는지 검증
    """
    try:
        data = pd.DataFrame({'x': x, 'y': y})
        test_result = grangercausalitytests(data[['y', 'x']],
                                            maxlag=max_lag,
                                            verbose=False)
        # 각 lag의 p-value 중 최소값 반환
        p_values = [test_result[lag][0]['ssr_ftest'][1]
                   for lag in range(1, max_lag+1)]
        return min(p_values)
    except:
        return 1.0

def calculate_score_advanced(corr, p_value, granger_p, lag):
    """
    정교한 공행성 점수 계산
    """
    # 1. 상관계수 점수 (0~50점)
    if abs(corr) >= 0.8:
        corr_score = 50
    elif abs(corr) >= 0.7:
        corr_score = 45
    elif abs(corr) >= 0.6:
        corr_score = 38
    elif abs(corr) >= 0.5:
        corr_score = 30
    else:
        corr_score = abs(corr) * 50

    # 2. 통계적 유의성 점수 (0~25점)
    if p_value < 0.001:
        sig_score = 25
    elif p_value < 0.01:
        sig_score = 20
    elif p_value < 0.05:
        sig_score = 12
    else:
        sig_score = 5

    # 3. Granger 인과성 점수 (0~20점)
    if granger_p < 0.001:
        granger_score = 20
    elif granger_p < 0.01:
        granger_score = 16
    elif granger_p < 0.05:
        granger_score = 10
    elif granger_p < 0.10:
        granger_score = 5
    else:
        granger_score = 0

    # 4. Lag 점수 (0~5점)
    if lag == 2 or lag == 3:
        lag_score = 5
    elif lag == 1 or lag == 4:
        lag_score = 4
    elif lag == 5:
        lag_score = 2
    else:
        lag_score = 0

    total_score = corr_score + sig_score + granger_score + lag_score
    return total_score

def find_comovement_pairs(
    pivot,
    residual_pivot,
    item_to_hs4,
    max_lag=6,
    min_nonzero=12,
    base_corr_threshold=0.3,
    base_score_threshold=35
):
    """
    HS4코드 유사도 반영, 적응적 임계값, Raw + Residual 종합 점수
    """
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)
    results = []

    for i, leader in tqdm(enumerate(items), total=len(items), desc="Finding pairs"):
        x_raw = pivot.loc[leader].values.astype(float)
        x_res = residual_pivot.loc[leader].values.astype(float)

        if np.count_nonzero(x_raw) < min_nonzero:
            continue

        leader_volatility = np.std(x_raw) / (np.mean(x_raw) + 1e-6)

        for follower in items:
            if follower == leader:
                continue

            y_raw = pivot.loc[follower].values.astype(float)
            y_res = residual_pivot.loc[follower].values.astype(float)

            if np.count_nonzero(y_raw) < min_nonzero:
                continue

            # HS4 유사도 체크
            hs4_leader = item_to_hs4.get(leader, "")
            hs4_follower = item_to_hs4.get(follower, "")
            same_hs4 = 1 if hs4_leader == hs4_follower else 0

            best_lag = None
            best_corr_raw = 0.0
            best_corr_res = 0.0
            best_p_value = 1.0

            # lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue

                corr_raw, p_raw = safe_corr_with_pvalue(x_raw[:-lag], y_raw[lag:])
                corr_res, p_res = safe_corr_with_pvalue(x_res[:-lag], y_res[lag:])
                combined_corr = 0.7 * abs(corr_raw) + 0.3 * abs(corr_res)

                if combined_corr > abs(best_corr_raw) * 0.7 + abs(best_corr_res) * 0.3:
                    best_corr_raw = corr_raw
                    best_corr_res = corr_res
                    best_lag = lag
                    best_p_value = min(p_raw, p_res)

            # 적응적 임계값
            adaptive_threshold = base_corr_threshold * (1 - 0.2 * min(leader_volatility, 1.0))
            combined_corr = 0.7 * abs(best_corr_raw) + 0.3 * abs(best_corr_res)

            if best_lag is not None and combined_corr >= adaptive_threshold:
                # Granger test
                granger_p = granger_test(x_raw, y_raw, max_lag=best_lag)

                # 종합 점수 계산
                score = calculate_score_advanced(
                    best_corr_raw, best_p_value, granger_p, best_lag
                )

                if same_hs4:
                    score += 5

                score += abs(best_corr_res) * 10

                if score >= base_score_threshold:
                    results.append({
                        "leading_item_id": leader,
                        "following_item_id": follower,
                        "best_lag": best_lag,
                        "max_corr": best_corr_raw,
                        "corr_residual": best_corr_res,
                        "same_hs4": same_hs4,
                        "combined_score": score
                    })

    pairs = pd.DataFrame(results)
    return pairs

# ============================================================
# 4. 페어 탐색 실행
# ============================================================

print("\n=== 공행성 쌍 탐색 시작 ===")
pairs = find_comovement_pairs(pivot, residual_pivot, item_to_hs4, max_lag=9, min_nonzero=12,
                               base_corr_threshold=0.3, base_score_threshold=35)

print(f"탐색된 공행성쌍 수: {len(pairs)}")

if len(pairs) == 0:
    print("경고: 공행성 쌍이 발견되지 않았습니다. 임계값을 낮춰보세요.")
else:
    pairs.to_csv(f"hs_pairs_{date_str}.csv", encoding='utf-8-sig', index=False)
    print(f"페어 저장 완료: hs_pairs_{date_str}.csv")

# ============================================================
# 5. 메타피처 생성
# ============================================================

print("\n===== 메타피처 생성 시작 =====")

PairsMeta_rows = []

for idx, row in tqdm(pairs.iterrows(), total=len(pairs), desc="Creating meta features"):
    leader = row["leading_item_id"]
    follower = row["following_item_id"]
    lag = int(row["best_lag"])
    corr_raw = float(row["max_corr"])

    # 시계열 준비
    ts_L_raw = pivot.loc[leader].fillna(0).values
    ts_F_raw = pivot.loc[follower].fillna(0).values
    ts_L_res = residual_pivot.loc[leader].fillna(0)
    ts_F_res = residual_pivot.loc[follower].fillna(0)

    # DTW (lag 반영)
    dtw_dist = np.nan
    if lag > 0 and lag < len(ts_L_raw):
        try:
            dtw_dist = dtw(ts_L_raw[:-lag], ts_F_raw[lag:])
        except:
            pass

    # Granger p-value
    granger_p = np.nan
    try:
        df_g = pd.DataFrame({"F": ts_F_raw, "L": ts_L_raw})
        if lag > 0 and len(df_g) > lag + 5:
            result = grangercausalitytests(df_g[["F", "L"]], maxlag=[lag], verbose=False)
            granger_p = result[lag][0]["ssr_ftest"][1]
    except:
        pass

    # residual volatility corr
    vol_corr = np.nan
    try:
        L_vol = ts_L_res.rolling(window=3).std().fillna(0)
        F_vol = ts_F_res.rolling(window=3).std().fillna(0)
        vol_corr = L_vol.corr(F_vol)
    except:
        pass

    # Corr scaling
    corr_scaled = np.log1p(abs(corr_raw)) * np.sign(corr_raw)

    PairsMeta_rows.append({
        "leading_item_id": leader,
        "following_item_id": follower,
        "f_corr_raw": corr_raw,
        "f_corr_scaled": corr_scaled,
        "f_lag": lag,
        "f_is_common": 0,  # 나중에 업데이트
        "f_dtw_distance_raw": dtw_dist,
        "f_granger_pvalue": granger_p,
        "f_volatility_corr_resid": vol_corr,
    })

pairs_meta = pd.DataFrame(PairsMeta_rows)
print(f"pairs_meta 생성 완료 → {pairs_meta.shape}")
pairs_meta.to_csv(f"hs_pairs_meta_{date_str}.csv", index=False, encoding="utf-8-sig")

# ============================================================
# 6. 학습 데이터 생성
# ============================================================

def build_training_data_A(pivot, pairs_meta, months_dt):
    """
    제출용 학습데이터 생성
    """
    months = months_dt
    n_months = len(months)
    rows = []

    for row in pairs_meta.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.f_lag)

        for t in range(lag, n_months - 1):
            ref_idx = t - lag
            target_month = months[t + 1]
            ref_month = months[ref_idx]

            # leader raw
            L_raw = pivot.loc[leader, ref_month]

            # leader last / last3 mean
            L_last = pivot.loc[leader, months[t]]
            L_last3 = pivot.loc[leader, months[max(0, t-2):t+1]].mean()

            # follower last / last3 mean
            F_last = pivot.loc[follower, months[t]]
            F_last3 = pivot.loc[follower, months[max(0, t-2):t+1]].mean()

            # follower mini-slope
            F_vals = pivot.loc[follower, months[max(0, t-2):t+1]].values
            if len(F_vals) >= 3:
                F_slope = (F_vals[-1] - F_vals[0]) / 3
            else:
                F_slope = 0

            rows.append({
                "leading_item_id": leader,
                "following_item_id": follower,
                "target_month": target_month,
                "value": pivot.loc[follower, target_month],

                # dynamic features
                "f_raw_value": L_raw,
                "f_leader_last": L_last,
                "f_leader_last3_mean": L_last3,
                "f_follower_last": F_last,
                "f_follower_last3_mean": F_last3,
                "f_follower_slope3": F_slope,

                # static meta
                "f_is_common": row.f_is_common,
                "f_lag": row.f_lag,
                "f_corr_scaled": row.f_corr_scaled,
                "f_dtw_distance_raw": row.f_dtw_distance_raw,
                "f_granger_pvalue": row.f_granger_pvalue,
                "f_volatility_corr_resid": row.f_volatility_corr_resid
            })
    return pd.DataFrame(rows)

feature_cols = [
    "f_raw_value",
    "f_leader_last",
    "f_leader_last3_mean",
    "f_follower_last",
    "f_follower_last3_mean",
    "f_follower_slope3",
    "f_is_common",
    "f_lag",
    "f_corr_scaled",
    "f_dtw_distance_raw",
    "f_granger_pvalue",
    "f_volatility_corr_resid"
]

# ============================================================
# 7. 모델 학습
# ============================================================

print("\n=== 학습 데이터 생성 ===")
dfA = build_training_data_A(pivot, pairs_meta, months_dt)
print(f'생성된 학습 데이터의 shape: {dfA.shape}')

if dfA.empty:
    print("오류: 학습 데이터가 없습니다.")
    submission = pd.DataFrame(columns=['leading_item_id', 'following_item_id', 'value'])
else:
    train_X = dfA[feature_cols].values
    train_y = dfA["value"].values

    print("\n=== 모델 학습 시작 ===")
    reg = LGBMRegressor(random_state=42, n_estimators=500, learning_rate=0.01)
    reg.fit(train_X, train_y)
    print("모델 학습 완료.")

    # Feature Importance
    importance_gain = reg.booster_.feature_importance(importance_type='gain')
    importance_split = reg.booster_.feature_importance(importance_type='split')

    idx = np.argsort(importance_gain)

    plt.figure(figsize=(8, 10))
    plt.barh(np.array(feature_cols)[idx], importance_gain[idx])
    plt.title("LightGBM Feature Importance (Gain)")
    plt.tight_layout()
    plt.savefig(f"feature_importance_gain_{date_str}.png", dpi=150)
    plt.close()

    plt.figure(figsize=(8, 10))
    plt.barh(np.array(feature_cols)[idx], importance_split[idx])
    plt.title("LightGBM Feature Importance (Split)")
    plt.tight_layout()
    plt.savefig(f"feature_importance_split_{date_str}.png", dpi=150)
    plt.close()

    print("Feature Importance 플롯 저장 완료.")

    # SHAP (선택사항)
    try:
        sample_size = min(2000, len(train_X))
        idx_sample = np.random.choice(len(train_X), sample_size, replace=False)
        X_sample = train_X[idx_sample]

        explainer = shap.TreeExplainer(reg)
        shap_values = explainer.shap_values(X_sample)

        shap.summary_plot(shap_values, X_sample, feature_names=feature_cols, plot_type="bar", show=False)
        plt.savefig(f"shap_summary_bar_{date_str}.png", dpi=150, bbox_inches='tight')
        plt.close()

        shap.summary_plot(shap_values, X_sample, feature_names=feature_cols, show=False)
        plt.savefig(f"shap_summary_{date_str}.png", dpi=150, bbox_inches='tight')
        plt.close()

        print("SHAP 플롯 저장 완료.")
    except Exception as e:
        print(f"SHAP 분석 실패: {e}")

    # ============================================================
    # 8. 예측
    # ============================================================

    print("\n=== 예측 시작 ===")
    preds = []
    t_last = len(months_dt) - 1

    for row in pairs_meta.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.f_lag)

        ref_idx = t_last - lag
        if ref_idx < 0:
            ref_idx = 0
        ref_month = months_dt[ref_idx]

        # dynamic
        L_raw = pivot.loc[leader, ref_month]
        L_last = pivot.loc[leader, months_dt[t_last]]
        L_last3 = pivot.loc[leader, months_dt[max(0, t_last-2):t_last+1]].mean()
        F_last = pivot.loc[follower, months_dt[t_last]]
        F_last3 = pivot.loc[follower, months_dt[max(0, t_last-2):t_last+1]].mean()

        F_vals = pivot.loc[follower, months_dt[max(0, t_last-2):t_last+1]].values
        if len(F_vals) >= 3:
            F_slope = (F_vals[-1] - F_vals[0]) / 3
        else:
            F_slope = 0

        Xrow = np.array([[
            L_raw, L_last, L_last3, F_last, F_last3, F_slope,
            row.f_is_common, row.f_lag, row.f_corr_scaled,
            row.f_dtw_distance_raw, row.f_granger_pvalue,
            row.f_volatility_corr_resid
        ]])

        pred = reg.predict(Xrow)[0]
        pred = int(max(0, round(pred)))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": pred
        })

    # 최종 제출 파일 저장
    file_name = f"hs_submission_{date_str}.csv"
    submission = pd.DataFrame(preds)
    submission.to_csv(file_name, index=False)
    print(f"제출용 예측 완료. {file_name} 저장됨.")
    print(f"제출 파일 shape: {submission.shape}")

print("\n=== 전체 프로세스 완료 ===")